In [1]:

##import packages
from tensorflow.keras import preprocessing, utils



2023-02-12 13:07:18.070762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-12 13:07:19.086802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-12 13:07:19.086876: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-12 13:07:22.278697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
from keras.backend import clear_session
import os
from gensim.models import Word2Vec
import re
import numpy as np
import yaml
import tensorflow as tf

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
##loading the path
dir_path = './data/'
files_list = os.listdir(dir_path + os.sep)
     

In [3]:
#question list
questions = list()
answers = list() #answers list from yml file
 
for filepath in files_list:#opens all yml file, read it, take the questions and answers from it. 
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)##load yml file
    conversations = docs['conversations'] ##take conversations key part
    for con in conversations:
        if len( con ) > 2 : ##if the length of con means under the conversation part all the text's len if >2 
            questions.append(con[0]) ##the frist part will be added
            replies = con[ 1 : ] ##after the first part added, the replies are added.
            ans = '' ##ans has empty list
            for rep in replies: ##take rep from replies list
                ans += ' ' + rep ##adding all text and make sentenses
            answers.append( ans ) ##append it in answers list
        elif len( con )> 1: ##if the con has more than 1 length
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list() ##tags appendings
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( ' ' + answers_with_tags[i] + ' ' )


tokenizer = preprocessing.text.Tokenizer()##convert text to sequence of integers 
tokenizer.fit_on_texts( questions + answers )##fit it
 
VOCAB_SIZE = len( tokenizer.word_index )+1 ##check the sequence of integers length
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

NameError: name 'yaml' is not defined

In [40]:


vocab = []
for word in tokenizer.word_index: ##then take the integer index of word which mostly common and maximum
  vocab.append(word)##append it in vocab

def tokenize(sentences):##tokenize function
  tokens_list = []##tokens_list
  vocabulary = []##vocabulary_list
  for sentence in sentences:
    sentence = sentence.lower()##convert each of the letter into lower
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)##replaced specific pattern to the sentence 
    tokens = sentence.split() ##split the words from sentence
    vocabulary += tokens##added the vocabulary
    tokens_list.append(tokens)##append it into the tokens_list
  return tokens_list, vocabulary
     


In [42]:
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )##convert into 2d Numpy array
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )##convert class vectors or integer into binary matrix
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(111, 26, 418)


In [41]:
tokenized_questions = tokenizer.texts_to_sequences( questions )##again convert into integer sequences
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)
     


# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )
     


(111, 9) 9
(111, 26) 26


In [43]:

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)##nlp related work
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer="Nadam", loss='categorical_crossentropy')

model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 9)]          0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 26)]         0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 9, 200)       83600       ['input_13[0][0]']               
                                                                                                  
 embedding_7 (Embedding)        (None, 26, 200)      83600       ['input_14[0][0]']               
                                                                                            

In [44]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=5, epochs=200 ,verbose=1) 
model.save( 'model.h5' )

Epoch 1/200
23/23 [==============================] - 14s 85ms/step - loss: 5.9039
Epoch 2/200
23/23 [==============================] - 2s 82ms/step - loss: 5.3047
Epoch 3/200
23/23 [==============================] - 2s 85ms/step - loss: 5.1200
Epoch 4/200
23/23 [==============================] - 2s 81ms/step - loss: 5.0116
Epoch 5/200
23/23 [==============================] - 2s 86ms/step - loss: 4.8657
Epoch 6/200
23/23 [==============================] - 2s 82ms/step - loss: 4.7682
Epoch 7/200
23/23 [==============================] - 2s 85ms/step - loss: 4.5592
Epoch 8/200
23/23 [==============================] - 2s 80ms/step - loss: 4.3795
Epoch 9/200
23/23 [==============================] - 2s 85ms/step - loss: 4.1918
Epoch 10/200
23/23 [==============================] - 2s 81ms/step - loss: 3.9813
Epoch 11/200
23/23 [==============================] - 2s 85ms/step - loss: 3.7992
Epoch 12/200
23/23 [==============================] - 2s 81ms/step - loss: 3.5327
Epoch 13/200
23/23 [====

In [45]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [46]:

def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [47]:
from keras.models import load_model

In [48]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    #empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

1/1 [==============================] - 0s 34ms/step
 software copying is a form of science which concerns itself with building hardware and software that replicates the functions of the human mind software software software that


KeyError: 'quit'

In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import time
from keras import regularizers
from keras.utils import plot_model
from keras.models import Model
from keras.optimizers import Adam
from keras.backend import clear_session
from keras.models import Sequential
from keras.layers import Flatten,Dense,Activation,Dropout,LSTM,Input,Conv1D,MaxPool1D,concatenate,Embedding
import tensorflow as tf
#from tensorflow.python.keras.optimizers import SGD
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('intents.json').read())
words=[]
classes = []
documents = []
ignore_letters = ['?', '!']
 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag'])) 
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('words.pk1','wb'))
pickle.dump(classes,open('classes.pk1','wb'))
training=[]
output_empty=[0]*len(classes)
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower())for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_Row=list(output_empty)
    output_Row[classes.index(document[1])]=1
    training.append([bag,output_Row])
random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])

#rnn

clear_session()
"""
model=Sequential()

model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
#sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
adam=Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=300,batch_size=5,verbose=1)
#model.save("voicebot_model.h5",hist)
#train_x=train_x.reshape(1,train_x.shape[0],train_x.shape[1])
#hist=model.fit(train_x,np.array(train_y),epochs=200,batch_size=5,verbose=1)
#model.save("voicebot_model_dnn1.h5",hist)
print('Done')
"""
##cnn-model


train_x=np.array(train_x)
train_y=np.array(train_y)
#train_x=train_x[:122]
#train_y=train_y[:122]
print(train_x.shape,train_y.shape)

2023-04-14 12:48:56.103629: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 12:48:56.335230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 12:48:56.335264: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-14 12:48:57.554267: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

(56, 111) (56, 14)


/tmp/ipykernel_4239/902772817.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [2]:
begin = time.time()
model=Sequential()
#model.add(Input(shape=(len(train_x[0]), )))

model.add(Embedding(len(train_x[0]), 2000, input_length = len(train_x[0])))

model.add(Conv1D(filters=2, kernel_size=3, activation='relu',kernel_initializer="he_normal"))
model.add(Conv1D(filters=4, kernel_size=3, activation='relu',kernel_initializer="he_normal"))
model.add(Conv1D(filters=12, kernel_size=3, activation='relu',kernel_initializer="he_normal"))

# concat1 = Concatenate()([conv1, conv2, conv3])
#concat1 = concatenate([conv1, conv2])
model.add(MaxPool1D(1))
model.add(Dropout(0.2))

model.add(Conv1D(filters=2, kernel_size=3, activation='relu',kernel_initializer="he_normal"))
model.add(Conv1D(filters=4, kernel_size=3, activation='relu',kernel_initializer="he_normal"))
model.add(Conv1D(filters=12, kernel_size=3, activation='relu',kernel_initializer="he_normal"))

#concat2 = concatenate([conv4, conv5])
model.add(MaxPool1D(1))
model.add(Dropout(0.2))

model.add(Conv1D(filters=12, kernel_size=3, activation='relu',kernel_initializer="he_normal"))
model.add(Flatten())
model.add(Dropout(0.2))

#Dense1 = Dense(128, activation='relu', kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(0.001))(dropout)
model.add(Dense(128, activation='relu', kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(len(train_y[0]), activation='softmax'))

#model = Model(inputs=input,outputs=output)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=300,batch_size=5,verbose=1)
end = time.time()
print(f"Total runtime of the program is {end - begin}")
#model.save("voicebot_model.h5",hist)
#train_x=train_x.reshape(1,train_x.shape[0],train_x.shape[1])
#hist=model.fit(train_x,np.array(train_y),epochs=200,batch_size=5,verbose=1)
#plot_model(model, show_shapes=True, to_file='model_graph.png')
#model.save("voicebot_model_cnn.h5",hist)
#print('Done')


2023-04-14 12:49:11.337937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-14 12:49:11.337978: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-14 12:49:11.338013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lenovoideapad32015ISK6c315432): /proc/driver/nvidia/version does not exist
2023-04-14 12:49:11.338462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 111, 2000)         222000    
                                                                 
 conv1d (Conv1D)             (None, 109, 2)            12002     
                                                                 
 conv1d_1 (Conv1D)           (None, 107, 4)            28        
                                                                 
 conv1d_2 (Conv1D)           (None, 105, 12)           156       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 105, 12)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 105, 12)           0         
                                                        